# Miscellaneous

In [1]:
# | default_exp misc

In [2]:
# | export
import time
from torch_snippets.logger import Debug, Excep, debug_mode, Info
from torch_snippets.markup2 import AD
from functools import wraps
from fastcore.basics import ifnone
from fastcore.foundation import L

[08/23/24 17:39:26] WARNING  Unable to load torch and dependent libraries from torch-snippets.                                                        ]8;id=216491;file:///Users/yeshwanth/Code/Personal/torch_snippets/torch_snippets/loader.py:108\loader.py]8;;\:]8;id=691599;file:///Users/yeshwanth/Code/Personal/torch_snippets/torch_snippets/loader.py:108#<module>:108\<module>:108]8;;\
                             Functionalities might be limited. pip install lovely-tensors in case there are torch related errors                                            

In [3]:
# | export
# | hide


class Timer:
    def __init__(self, N, smooth=True, mode=1):
        "print elapsed time every iteration and print out remaining time"
        "assumes this timer is called exactly N times or less"
        self.tok = self.start = time.time()
        self.N = N
        self.ix = 0
        self.smooth = smooth
        self.mode = mode
        # 0 = instant-speed, i.e., time remaining is a funciton of only the last iteration
        # Useful when you know that each loop takes unequal time (increasing/decreasing speeds)
        # 1 = average, i.e., time remaining is a function of average of all iterations
        # Usefule when you know on average each loop or a group of loops take around the same time

    def __call__(self, ix=None, info=None):
        ix = self.ix if ix is None else ix
        info = "" if info is None else f"{info}\t"
        tik = time.time()
        elapsed = tik - self.start

        if self.mode == 0:
            ielapsed = tik - self.tok
            ispeed = ielapsed
            iremaining = (self.N - (ix + 1)) * ispeed

            iunit = "s/iter"
            if ispeed < 1:
                ispeed = 1 / ispeed
                iunit = "iters/s"
            iestimate = iremaining + elapsed
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {iremaining:.2f}s remaining - {ispeed:.2f} {iunit})"

        else:
            speed = elapsed / (ix + 1)
            remaining = (self.N - (ix + 1)) * speed
            unit = "s/iter"
            if speed < 1:
                speed = 1 / speed
                unit = "iters/s"
            estimate = remaining + elapsed
            # print(f'N={self.N} e={elapsed:.2f} _rem={self.N-(ix+1)} r={remaining:.2f} s={speed:.2f}\t\t')
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {remaining:.2f}s remaining - {speed:.2f} {unit})"

        print(
            _info + " " * 10,
            end="\r",
        )
        self.ix += 1
        self.tok = tik


def track2(iterable, *, total=None):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops

In [4]:
def track2(iterable, *, total=None, timer_mode=1):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total, mode=timer_mode)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops


l = list(range(10, 0, -1))
fact = 10
t = sum(l) / fact
for i in track2(l):
    time.sleep(i / fact)
    print()


1/10 (1.01s - 9.05s remaining - 1.01 s/iter)          
2/10 (1.91s - 7.63s remaining - 1.05 iters/s)          
3/10 (2.71s - 6.33s remaining - 1.11 iters/s)          
4/10 (3.42s - 5.12s remaining - 1.17 iters/s)          
5/10 (4.02s - 4.02s remaining - 1.24 iters/s)          
6/10 (4.52s - 3.01s remaining - 1.33 iters/s)          
7/10 (4.92s - 2.11s remaining - 1.42 iters/s)          
8/10 (5.23s - 1.31s remaining - 1.53 iters/s)          
9/10 (5.43s - 0.60s remaining - 1.66 iters/s)          


Use timer as a standalone class so you have full control on when to call a lap (most useful in while loops)...

In [5]:
N = 100
t = Timer(N)
info = None

for i in range(N):
    time.sleep(0.1)
    t(info=info)  # Lap and present the time
    if i == 50:
        print()
        info = f"My Info: {i*3.122}"

51/100 (5.28s - 5.07s remaining - 9.66 iters/s)          


... or use track2 to directly track a loop

In [6]:
N = 100
info = None

for i in (tracker := track2(range(N), total=N)):
    time.sleep(0.1)
    info = f"My Info: {i*3.122:.2f}"
    if i == N // 2:
        print()
    if i >= N // 2:
        tracker.send(info)

50/100 (5.20s - 5.20s remaining - 9.61 iters/s)          


## Warning! NEVER RUN `tracker.send(None)` as this will skip variables silently

In [7]:
# | export
# | hide


def summarize_input(args, kwargs, outputs=None):
    o = AD(args, kwargs)
    if outputs is not None:
        o.outputs = outputs
    return o.summary()


def timeit(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Debug(f"{time.time() - s:.2f} seconds to execute `{func.__name__}`")
        return o

    return inner


def io(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        info = f"""
{time.time() - s:.2f} seconds to execute `{func.__name__}`
{summarize_input(args=args, kwargs=kwargs, outputs=o)}
        """
        Debug(info, depth=1)
        return o

    return inner

In [8]:
@io
@timeit
def foo(a, b=None):
    if b is None:
        return a + 1
    else:
        time.sleep(2)
        return a + b


with debug_mode():
    foo(10)
    foo(10, b=20)

[08/23/24 17:39:53] DEBUG    0.00 seconds to execute `foo`                                                                                            ]8;id=645548;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1296653500.py:16\1296653500.py]8;;\:]8;id=353152;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1296653500.py:16#inner:16\inner:16]8;;\

                    DEBUG                                                                                                                          ]8;id=824600;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/2064456364.py:12\2064456364.py]8;;\:]8;id=898659;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/2064456364.py:12#<module>:12\<module>:12]8;;\
                             0.00 seconds to execute `inner`                                                                                                                
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                             outputs - 11 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

[08/23/24 17:39:55] DEBUG    2.01 seconds to execute `foo`                                                                                            ]8;id=621942;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1296653500.py:16\1296653500.py]8;;\:]8;id=222460;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1296653500.py:16#inner:16\inner:16]8;;\

                    DEBUG                                                                                                                          ]8;id=179073;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/2064456364.py:13\2064456364.py]8;;\:]8;id=325734;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/2064456364.py:13#<module>:13\<module>:13]8;;\
                             2.01 seconds to execute `inner`                                                                                                                
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                               b - 20 (🏷️ int)                                                                                                                               
                             outputs - 30 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

In [32]:
# | export
# | hide


def tryy(
    func=None,
    *,
    output_to_return_on_fail=None,
    silence_errors=False,
    print_traceback=False,
    store_errors: bool = True,
):
    def decorator(f):
        if isinstance(store_errors, bool) and store_errors:
            error_store = []
        elif isinstance(store_errors, (list, L)):
            error_store = store_errors

        def inner(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except Exception as e:
                if not print_traceback:
                    tb = f"{type(e).__name__}: {str(e)}"
                else:
                    import traceback

                    tb = traceback.format_exc()
                if not silence_errors:
                    Excep(
                        f"Error for `{f.__name__}` with \n{summarize_input(args, kwargs)}\n{tb}"
                    )
                if store_errors is not None:
                    error_store.append(
                        AD(
                            func=f.__name__,
                            args=args,
                            kwargs=kwargs,
                            tb=tb,
                            err_type=type(e).__name__,
                        )
                    )
                return output_to_return_on_fail

        inner.F = f  # Store reference to the original function
        inner.error_store = error_store

        def error_summary():
            import pandas as pd

            return pd.DataFrame([e.dict() for e in error_store])

        inner.error_summary = error_summary
        return inner

    if callable(func):
        return decorator(func)
    return decorator

## Try Catch with a single line

In [33]:
@tryy
def do(a, b, c):
    return 1 / 0


x = do(1, 2, c=10)
assert x is None  # tryy returns None by default

[08/23/24 17:48:03] ERROR    Error for `do` with                                                                                                      ]8;id=270527;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30\1057038067.py]8;;\:]8;id=157819;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            

Use your own default on failure

In [34]:
@tryy(output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

[08/23/24 17:48:04] ERROR    Error for `do` with                                                                                                      ]8;id=550994;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30\1057038067.py]8;;\:]8;id=605547;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            

'😔'

Optionally print the full stacktrace if needed

In [35]:
@tryy(print_traceback=True, output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

[08/23/24 17:48:06] ERROR    Error for `do` with                                                                                                      ]8;id=430447;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30\1057038067.py]8;;\:]8;id=852031;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/1057038067.py", line 21, in inner                                     
                                 return f(*args, **kwargs)                                                                                                                  
                                        ^^^^^^^^^^^^^^^^^^                                                                                                                  
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_28490/580638143.py", line 3, in do                                          
                                 return 1 / 0                                                                                                                               
                                        ~~^~~                                                                                                                               
                             ZeroDivisionError: division by zero                                                                                                            
                                                                                                                                                                            

'😔'

You can also silence the errors completely

In [36]:
@tryy(silence_errors=True, output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

'😔'

You can collect all your errors in a list

In [37]:
import random

errors = []


@tryy(silence_errors=True, store_errors=errors)
def do(a, b, c):
    if random.randint(0, 100) < 50:
        return 1 / 0
    else:
        raise NotImplementedError("🤔")


for _ in range(4):
    do(1, random.randint(0, 10), c=random.randint(0, 100))

print(errors)

[
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 8 (🏷️ int)
kwargs
  c - 83 (🏷️ int)
tb - ZeroDivisionError: division by zero (🏷️ str)
err_type - ZeroDivisionError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 9 (🏷️ int)
kwargs
  c - 88 (🏷️ int)
tb - ZeroDivisionError: division by zero (🏷️ str)
err_type - ZeroDivisionError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 3 (🏷️ int)
kwargs
  c - 59 (🏷️ int)
tb - NotImplementedError: 🤔 (🏷️ str)
err_type - NotImplementedError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 6 (🏷️ int)
kwargs
  c - 6 (🏷️ int)
tb - NotImplementedError: 🤔 (🏷️ str)
err_type - NotImplementedError (🏷️ str)

```
]


There's onlly one usecase where you would want to send in a list by yourself - when you want to append your errors to an existing list. The sensible default is to always store the errors, especially because this is a debugging tool.

Just access all the errors in a dataframe like so

In [49]:
import random

random.seed(10)
errors = []


@tryy(silence_errors=True, store_errors=errors, output_to_return_on_fail="😔")
def do(a, b, c):
    if c < 50:
        return 1 / 0
    else:
        raise NotImplementedError("🤔")


for _ in range(4):
    do(1, random.randint(0, 10), c=random.randint(0, 100))

do.error_summary()

,func,args,kwargs,tb,err_type
0,do,"[1, 9]",{'c': 4},ZeroDivisionError: division by zero,ZeroDivisionError
1,do,"[1, 6]",{'c': 61},NotImplementedError: 🤔,NotImplementedError
2,do,"[1, 9]",{'c': 1},ZeroDivisionError: division by zero,ZeroDivisionError
3,do,"[1, 3]",{'c': 59},NotImplementedError: 🤔,NotImplementedError


and the actual list of errors like so

In [52]:
do.error_store

[
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 9 (🏷️ int)
 kwargs
   c - 4 (🏷️ int)
 tb - ZeroDivisionError: division by zero (🏷️ str)
 err_type - ZeroDivisionError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 6 (🏷️ int)
 kwargs
   c - 61 (🏷️ int)
 tb - NotImplementedError: 🤔 (🏷️ str)
 err_type - NotImplementedError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 9 (🏷️ int)
 kwargs
   c - 1 (🏷️ int)
 tb - ZeroDivisionError: division by zero (🏷️ str)
 err_type - ZeroDivisionError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 3 (🏷️ int)
 kwargs
   c - 59 (🏷️ int)
 tb - NotImplementedError: 🤔 (🏷️ str)
 err_type - NotImplementedError (🏷️ str)
 
 ```]

Finally, you want to run the function (without try) to reproduce the error and actually start debugging. Just use the `.F` attribute to access the original function that you created

In [51]:
ix = 2
data = do.error_store[ix]
do.F(*data.args, **data.kwargs)

ZeroDivisionError: division by zero

In [13]:
def deco(decorator):
    @wraps(decorator)
    def wrapper(*args, **kwargs):
        def real_decorator(func):
            @wraps(func)
            def inner_wrapper(*fargs, **fkwargs):
                return decorator(func, *fargs, **fkwargs)

            return inner_wrapper

        if len(args) == 1 and callable(args[0]) and not kwargs:
            # Case when B is used without arguments
            return real_decorator(args[0])
        else:
            # Case when B is used with arguments
            def custom_decorator(func):
                return decorator(func, **kwargs)

            return custom_decorator

    return wrapper

In [14]:
@deco
def B(func, *args, deco_param="default", **kwargs):
    print("B", deco_param, print("args", *args, "kwargs", **kwargs))
    return func(*args, **kwargs)

In [15]:
@B
def C(a, b, c):
    print("C")
    return a + (b * c)


C(1, 2, 3)

args 1 2 3 kwargs
B default None
C


7

In [16]:
@B(deco_param="new_param")
def C(a, b, c):
    return a + (b * c)


# Testing
print(C(1, 2, 3))  # Outputs 'new_param' then the result 7

args kwargs
B new_param None


TypeError: C() missing 3 required positional arguments: 'a', 'b', and 'c'